<a href="https://colab.research.google.com/github/mlindsey19/evol-comp-two/blob/master/EvolComp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy as np

N = 10 # must be even for crossover
RUNS = 30
LOCI = 3
LCBD = (-1.0, 5.0) #loci bounds
#print format -> #.###
float_formatter = lambda x: "%.8f" % x
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )


In [0]:
pen = 2
penFlag = 1

def fitness( individual ):
  fit = 0
  for x in individual:
    fit +=  pen * x**2 if ( penFlag and ( x > 5.0 or x < -1.0 ) ) else x**2
  return fit

def fitVector(g, n):
  fit = np.empty(n)
  for i in range(n):
    fit[ i ] = fitness( g[ i ] )
  return fit


def sortFit( g, n = N ):
  temp = np.c_[ g, fitVector( g, n ) ]
  return temp[ temp[ : , 3 ].argsort() ]


In [0]:
Pc = 0.8
alpha = 0.7
shuffFlag = 0

def crossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      gen[ j ][ 0 ], gen[ j + 1 ][ 0 ] = gen[ j + 1 ][ 0 ],  gen[ j ][ 0 ]
#  return gen

def arithmeticCrossover(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pc ):
      j = 2 * i
      x = gen[ j + 1 ][ 0 ] + gen[ j ][ 0 ]
      gen[ j ][ 0 ], gen[ j + 1 ][ 0 ] = ( 1 - alpha ) * x, alpha * x
  #    return gen

def arithmeticCrossPerLoci(  ):
  if ( shuffFlag ):
    np.random.shuffle( gen )
  for i in range( N // 2 ):
    for h in range( 3 ):        
      j = 2 * i
      if ( np.random.uniform( 0, 1 ) < Pc ):
        x = gen[ j + 1 ][ h ] + gen[ j ][ h ]
        gen[ j ][ h ], gen[ j + 1 ][ h ] = ( 1 - alpha ) * x, alpha * x
 # return gen
    
  
 

***mr*** mutation range
***ub*** upper bound

The upper bound is calculated so that numbers selected at random fall in an appropriate 
range of mutation ***P*** % of the time 

In [0]:
Pm = 0.1
mr = 0.75

def mut():
  mtmt = np.empty( ( N, LOCI) )
  ub = ( 2 * mr / Pm ) - mr
  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( (0 - mr), ub )
      if ( m < mr ):
        mtmt[i][j] = m
      else:
        mtmt[i][j] = 0  
  return mtmt

def gausIndi():
  mtmt = np.empty( ( N, LOCI) )

  for i in range ( N ):
    sd = np.std(gen[i])
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m < Pm ):
        mtmt[i][j] = np.random.normal(0, sd)
      else:
        mtmt[i][j] = 0   
  return mtmt

def gausPop():
  sd = np.std(gen)
  mtmt = np.random.normal( 0.0, sd, ( N, LOCI) )

  for i in range ( N ):
    for j in range(LOCI):
      m = np.random.uniform( 0 , 1 )
      if ( m > Pm ):
            mtmt[i][j] = 0  
  return mtmt

def mutation(  ):
  global gen
  gen = gen + mut()

def gpmutation(  ):
  global gen
  gen = gen + gausPop()
def gimutation(  ):
  global gen
  gen = gen + gausIndi()


def mutationWithClamp(  ):
  global gen
  ngen = gen + mut()
  for rw in range( N ):
    for cl in range( LOCI ):  
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] > -1 ) else -1
        ngen[rw][cl] = ngen[rw][cl] if ( ngen[rw][cl] < 5 ) else 5
  gen = ngen

  



In [13]:
Psbt = 0.4

def selectionBT(  ):
  global gen
  ngen = np.empty( ( N, LOCI) )
  for i in range(N):
    x = gen[ random.randint( 0, N - 1 ) ]
    y = gen[ random.randint( 0, N - 1 ) ]
    #print( x , y)
    if( fitness( x ) > fitness( y ) ):
      x = y
    ngen[ i ] = x
  gen = ngen

selectionBT()

print( gen )

[[0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]]


In [12]:
min = .5
max = 2 - min
Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
pmap =np.empty(N)
for i in range(N):
  if (i == 0):
    pmap[ i ] = PlrMap[ i ]
  else:
    pmap[ i ] = PlrMap[ i ] + pmap[ i - 1 ]


def selectionLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

en = 2  # number of best fit guarantee propogate  

def selectionEliteLR(  ):
  global gen
  ngen = np.empty( (N, LOCI) )
  temp = sortFit( gen )
  for i in range( en ):
    ngen[ i ] = temp[ i, : 3]
  for i in range( en, N ):
    s = np.random.uniform( 0, 1 )
    for idx,c in enumerate(pmap):
      if( s <= c ):
        #print(s, c , 9 - idx)
        ngen[ i ] = temp[ 9-idx, : 3 ] 
        break
  gen = ngen

selectionLR()
print(gen)
selectionEliteLR()

print( gen )

[[0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]]
[[0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]]


In [15]:
tn = 8 # number to keep

firstRunFlag = 1
if ( firstRunFlag ):
  firstRunFlag = 0
  prevgen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
  
def selectionTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  temp = sortFit( gen )
  for i in range( tn ):
    ngen[ i ] = temp[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

def selectionFibTrunc(  ):
  global gen
  ngen = np.empty( ( N, LOCI ) )
  global prevgen
  temp = gen
  union = np.append(gen, prevgen, axis = 0)
  union = sortFit(union, n=(2 * N) )
  prevgen = temp
  for i in range( tn ):
    ngen[ i ] = union[ i, : 3 ]
  for i in range ( tn, N ):
    ngen [ i ] = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( 1, LOCI ) )
  gen = ngen

selectionFibTrunc()
print(gen)


[[0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.93940923 0.63069757 0.58412441]
 [0.82278287 3.93605744 3.97061021]
 [2.34071517 1.07328978 3.25365569]]


In [0]:
for i in range(50):
  mutation()
  mutationWithClamp()
  arithmeticCrossPerLoci()
  arithmeticCrossover()
  arithmeticCrossPerLoci()
  crossover()
  selectionBT() 
  selectionTrunc() 
  selectionFibTrunc()
  selectionLR()
  selectionEliteLR()
print (gen)



In [19]:


mutList = [ gimutation, mutation, gpmutation]
crossList = [ arithmeticCrossover, arithmeticCrossPerLoci, crossover ]
selList = [ selectionBT, selectionTrunc, selectionFibTrunc, selectionLR, selectionEliteLR ]
avg = np.empty((3,3,5))
best = np.empty((3,3,5))
stdv = np.empty((3,3,5))
def evol():
  global gen
  global avg
  global best 
  global stdv
  bst = 0.0
  avgVector = np.empty(30)
  for c, cross in enumerate( crossList ):
    for m, mut in enumerate( mutList ):
      for s, sel in enumerate( selList ):
          for i in range( 30 ):
            gen = np.random.uniform( LCBD[ 0 ], LCBD[ 1 ], ( N, LOCI ) )
            for _ in range( 50 ):
              sel()
              cross()
              mut()
            bst = np.amin(fitVector(gen, N))
            avgVector[ i ] = np.mean(fitVector(gen, N))
          best[c][m][s] = bst if (bst < best[c][m][s] ) else best[c][m][s]
          avg[c][m][s] = np.mean(avgVector)
          stdv[c][m][s] = np.std(avgVector)



evol()

print( avg )
print( best )
print( stdv )

[[[0.00370149 6.03961570 4.18287573 0.07324644 0.00054557]
  [0.13208465 5.88046186 4.31291227 0.26353355 0.08903345]
  [0.00313995 7.63579423 4.24961494 0.50201763 0.00100705]]

 [[0.00000000 4.07359997 5.02072068 0.00000957 0.00000000]
  [0.07237357 5.51277470 4.20471691 0.10546637 0.08229169]
  [0.00000000 4.99948197 4.97100329 0.00002161 0.00000000]]

 [[0.11811740 6.02574518 4.72045009 0.08545933 0.00134773]
  [0.10924981 5.66187650 4.33625886 0.34761807 0.09804457]
  [0.00444314 8.88115196 5.07310551 0.07014569 0.00082361]]]
[[[0.00000484 0.00000000 0.00000000 0.00000000 0.00000000]
  [0.00231570 0.08467657 0.00000394 0.00000000 0.00000000]
  [0.00000017 0.00000000 0.00000080 0.00000000 0.00000000]]

 [[0.00000000 0.00086492 0.00000000 0.00000000 0.00000000]
  [0.00040014 0.07668963 0.00000000 0.00000000 0.00000000]
  [0.00000000 0.20336110 0.00000000 0.00000000 0.00000000]]

 [[0.00099163 0.05625354 0.00000000 0.00000000 0.00000000]
  [0.00408374 0.33387187 0.00000000 0.00000000